In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import max_error
import numpy as np

In [ ]:
data = pd.read_pickle(r"../data/B00020S.pkl")
data['Date'] = pd.to_datetime(data['Date'])

# TODO
 - znaleźć prawidłowe offset_values — Max **DONE**
 - wykresy błędów — Adam
 - nałożyć wartość bezwzględną na histogram błędu bezwzględnego — Adam
 - upiększyć wykresy — Max
 - dodać do temp kolumnę z datami — Max
 - może jakiś opis matematyczny w markdownie tej regresji czy coś — Max albo Adam
 - zautomatyzować dobór offset_values - Max

In [ ]:
# Stacje
stations = ['GŁOGÓW', 'ŚCINAWA', 'MALCZYCE', 'BRZEG DOLNY', 'OŁAWA', 'BRZEG', 'KRAPKOWICE', 'RACIBÓRZ-MIEDONIA', 'KRZYŻANOWICE', 'OLZA', 'CHAŁUPKI']

# Lista różnic w dniach między Głogowem a kolejną stacją
offset_values = [0, 1, 1, 1, 2, 2, 3, 3, 3, 3]

# Rok, od którego chcemy trenować i testować model
start_year = 2019

# Grupowanie po dniach i stacjach
data_grouped = data.groupby(['Date', 'Station'])['B00020S'].mean().reset_index()

# Osobny dataframe dla każdej stacji i tylko rekordy od danego roku
station_datas = [data_grouped[(data_grouped['Station'] == station) & (data_grouped['Date'].dt.year >= start_year)].reset_index() for station in stations]

# Połączenie kolumn z poziomem wody z każdej stacji w jeden dataframe
temp = pd.concat([station_data['B00020S'] for station_data in station_datas], axis='columns').reset_index(drop=True)

# Zmiana nazw kolumn
temp.columns = stations

# Przesunięcie każdej kolumny o odpowiednią liczbę dni
for i, col in enumerate(temp.columns):
    temp[col] = temp[col].shift(periods=-offset_values[i])

# Usunięcie NA, możliwe, że później można to dopracować
temp = temp.dropna()

# Zmienne służące do podziału zbioru danych na zbiór treningowy i zbiór testujący
test_proportion = 0.2
train_proportion = 1 - test_proportion
split_point = int(len(temp) * train_proportion)

# Podział na zbiór treningowy i zbiór testowy
train_data, test_data = temp.iloc[:split_point], temp.iloc[split_point:]

# Zmienne niezależne
x = train_data.iloc[:, 1:]
x = sm.add_constant(x)

# Zmienna niezależna
y = train_data.iloc[:, 0]

# Dopasowanie modelu
model = sm.OLS(y, x).fit()

# Podsumowanie
print(model.summary())

In [ ]:
# Dataframe z resztami modelu
residuals = pd.DataFrame(model.resid)

# Wykres reszt
residuals.plot()

# Wykres gęstości jądra reszt / rozkład prawdopodobieństwa reszt
residuals.plot(kind='kde')

# Statystyki opisowe reszt
print(residuals.describe())

In [ ]:
# Przewidywanie wartości na podstawie modelu
model.predict(x)

# To samo co wyżej, ale ręcznie
test_values = model.params['const'] + sum(model.params[param] * test_data[param] for param in model.params.index[1:])

# Tymczasowa oś X, trzeba będzie zmienić na prawdziwą datę
date_train = np.arange(0, len(train_data))
date_test = np.arange(len(train_data), len(train_data) + len(test_data))

# Ustawienie rozmiaru całego wykresu (jednostki są w calach
plt.figure(figsize=(36, 16))

# Prawdziwe dane treningowe
plt.plot(date_train, train_data.iloc[:,0], 'b', label='faktyczne dane', linewidth=3)

# Dane treningowe modelu
plt.plot(date_train, model.predict(), 'r', label='model', linewidth=3)

# Prawdziwe dane testowe
plt.plot(date_test, test_data.iloc[:, 0], 'g', label='faktyczne dane (test)', linewidth=3)

# Dane testowe modelu
plt.plot(date_test, test_values, 'm', label='model (test)', linewidth=3)

# Granice osi X i Y wykresu
plt.xlim(0, len(date_train) + len(date_test))
plt.ylim(0, 1000)

# Parametry wykresu
plt.legend(loc='upper right', fontsize=20)
plt.title("Coś tam coś tam, nwm Adam nazwie", fontsize=30)
plt.xlabel('Miesiąc (xD)', fontsize=15)
plt.ylabel('Poziom wody (cm)', fontsize=15)
plt.grid()
plt.show()

In [ ]:
# Prawdziwe dane testowe
y_true = test_data.iloc[:, 0]

# Średni błąd kwadratowy
mse = mean_squared_error(y_true=y_true, y_pred=test_values, squared=True)

# Pierwiastek błędu kwadratowego
rmse = mean_squared_error(y_true=y_true, y_pred=test_values, squared=False)

# Błąd procentowy średniokwadratowy
mape = mean_absolute_percentage_error(y_true=y_true, y_pred=test_values)

# Maksymalny błąd
max_error = max_error(y_true=y_true, y_pred=test_values)

# Wyświetlenie wszystkich powyższych metryk
print('Test MSE: %.3f' % mse)
print('Test RMSE: %.3f' % rmse)
print('Test MAPE: %.3f' % mape)
print('Test max error: %.3f' % max_error)

In [ ]:
# Funkcja do liczenia korelacji Pearsona między dwiema stacjami
def correlation_between_stations(station_1, station_2, lag):
    station_1_id = stations.index(station_1.upper())
    station_2_id = stations.index(station_2.upper())
    correlation = lagged_dfs[lag][station_1_id]['B00020S'].corr(lagged_dfs[0][station_2_id]['B00020S'])
    return round(correlation, 3)

# Stacje
stations = ['GŁOGÓW', 'ŚCINAWA', 'MALCZYCE', 'BRZEG DOLNY', 'OŁAWA', 'BRZEG', 'KRAPKOWICE', 'RACIBÓRZ-MIEDONIA', 'KRZYŻANOWICE', 'OLZA', 'CHAŁUPKI']

# Grupowanie po dniach i stacjach
data_grouped = data.groupby(['Date', 'Station'])['B00020S'].mean().reset_index()

# Maksymalny lag
max_lag = 7

# Lista od 0 do max_lag
lags = range(max_lag + 1)

# Dwuwymiarowa lista zlagowanych dataframe'ów stacji
# Dostęp do wybranej stacji i lagu: lagged_dfs[lag][id_stacji np. 0 dla Głogowa]
start_date = '2008-01-08'
end_date = '2023-09-30'
lagged_dfs =[[data_grouped[(data_grouped['Date']
                .between(
                    pd.to_datetime(start_date) - pd.DateOffset(days=lag),
                    pd.to_datetime(end_date) - pd.DateOffset(days=lag)
                ))&(data_grouped['Station'] == station)].reset_index(drop=True) for station in stations]for lag in lags
             ]
# Lista stacji, dla których chcemy obliczyć korelację Pearsona względem Głogowa
stations_to_calculate_corr = ['ŚCINAWA', 'MALCZYCE']

# Wyświetlanie korelacji Pearsona
for station in stations_to_calculate_corr:
    print(f"\nWspółczynnik korelacji Pearsona liczony na podstawie poziomu wody w stacjach: {station.capitalize()} - Głogów")
    for lag in lags:
        print(f"Lag: {lag}, p = {correlation_between_stations(station, 'Głogów', lag)}")